In [213]:
import numpy as np

In [214]:
# def matriz_espelhada_aleatoria(dimensao, valor_min, valor_max):
#     # Cria uma matriz com valores aleatórios
#     matriz = np.random.randint(valor_min, valor_max/2, size=(dimensao, dimensao))
    
#     # Define a diagonal principal como zero
#     np.fill_diagonal(matriz, 0)

#     # Espelha a matriz
#     matriz_simetrica = (matriz + matriz.T)
    
#     return matriz_simetrica

# # Dimensão da matriz distancia entre as cidades, com o valor mínimo e máximo das distancias
# dimensao = num_cities
# valor_min = 1
# valor_max = 50

# # Distâncias entre as cidades 
# distances = matriz_espelhada_aleatoria(dimensao, valor_min, valor_max)

# print(distances)

In [215]:
distances = np.loadtxt('matriz_adjacencia.txt', dtype=int)

print(distances)

[[ 0 21 25 ... 13 25 21]
 [21  0 40 ... 16  7 28]
 [25 40  0 ... 33 13 17]
 ...
 [13 16 33 ...  0 12 18]
 [25  7 13 ... 12  0 33]
 [21 28 17 ... 18 33  0]]


In [216]:
# Parâmetros do ACO
num_cities = 100  # Número de cidades
num_ants = 15  # Número de formigas, definindo o número de soluções construídas por iteração
num_iterations = 500 # Número de iterações, quanto maior, mais tempo o algoritmo tem para explorar o espaço de soluções, mas também mais tempo para convergir
alpha = 1.0  # Importância dos feromônios
beta = 5.0  # Importância da visibilidade (inverso da distância)
evaporation_rate = 0.4  # Taxa de evaporação dos feromônios
initial_pheromone = 1.0  # Valor inicial dos feromônios

In [217]:
# Inicialização da matriz de feromônios
pheromones = np.full((num_cities, num_cities), initial_pheromone) # Inicializa a matriz de feromônios com o valor inicial

# Função para calcular a visibilidade (inverso da distância)
def calculate_visibility(distances):
    visibility = 1 / distances
    visibility[distances == 0] = 0  # Evitar divisão por zero, definindo visibilidade para distâncias iguais a zero como zero. Para cidade i, visibilidade[i, i] = 0
    return visibility

# Função para escolher a próxima cidade com base nas probabilidades
def choose_next_city(pheromones, visibility, current_city, visited):    
    probabilities = []
    for city in range(num_cities):
        if city not in visited:
            pheromone = pheromones[current_city, city] ** alpha
            vis = visibility[current_city, city] ** beta
            probabilities.append(pheromone * vis)
        else:
            probabilities.append(0)
    probabilities = probabilities / np.sum(probabilities)
    next_city = np.random.choice(range(num_cities), p=probabilities)
    return next_city

# Função principal do ACO para o TSP
def ant_colony_optimization(distances, num_ants, num_iterations, evaporation_rate):
    num_cities = distances.shape[0] 
    pheromones = np.full((num_cities, num_cities), initial_pheromone)
    visibility = calculate_visibility(distances)

    best_path = None
    best_path_length = float('inf')

    for iteration in range(num_iterations):
        all_paths = []
        for ant in range(num_ants):
            path = [np.random.randint(num_cities)]
            while len(path) < num_cities:
                next_city = choose_next_city(pheromones, visibility, path[-1], path)
                path.append(next_city)
            path.append(path[0])  # Retornar à cidade de origem
            all_paths.append(path)

        for path in all_paths:
            path_length = sum(distances[path[i], path[i+1]] for i in range(num_cities))
            if path_length < best_path_length:
                best_path_length = path_length
                best_path = path

            for i in range(num_cities):
                pheromones[path[i], path[i+1]] += 1.0 / path_length

        pheromones *= (1 - evaporation_rate)

    return best_path, best_path_length

# Executar o ACO
best_path, best_path_length = ant_colony_optimization(distances, num_ants, num_iterations, evaporation_rate)
print(f"Melhor caminho encontrado: {best_path}")
print(f"Comprimento do melhor caminho: {best_path_length}")

C:\Users\lcex\AppData\Local\Temp\ipykernel_6632\725916924.py:6: RuntimeWarning: divide by zero encountered in divide
  visibility = 1 / distances


Melhor caminho encontrado: [39, 48, 24, 29, 37, 84, 87, 25, 45, 83, 81, 43, 76, 69, 34, 47, 58, 41, 97, 5, 31, 95, 78, 93, 85, 53, 40, 35, 32, 64, 79, 22, 75, 92, 26, 3, 27, 82, 55, 86, 17, 62, 4, 70, 91, 61, 2, 23, 96, 12, 38, 30, 10, 67, 94, 8, 59, 89, 33, 63, 52, 54, 1, 18, 11, 49, 16, 60, 21, 72, 28, 68, 19, 9, 66, 36, 90, 56, 20, 50, 51, 0, 71, 80, 65, 42, 6, 73, 57, 13, 46, 98, 14, 88, 77, 7, 99, 74, 15, 44, 39]
Comprimento do melhor caminho: 618
